<a href="https://colab.research.google.com/github/rishabkolan/UFC-Fight-Duration-Prediction/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **UFC Fight Duration Prediction**

# 1. Data Preprocessing

In [1]:
#Importing the Libraries

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from google.colab import drive

In [2]:
#Creating the dataframe

drive.mount('/content/drive')

file_path = '/content/drive/MyDrive/Classroom/large_dataset.csv'
df = pd.read_csv(file_path)

Mounted at /content/drive


In [3]:
df.head(5)

,event_name,r_fighter,b_fighter,winner,weight_class,is_title_bout,gender,method,finish_round,total_rounds,...,weight_diff,reach_diff,SLpM_total_diff,SApM_total_diff,sig_str_acc_total_diff,td_acc_total_diff,str_def_total_diff,td_def_total_diff,sub_avg_diff,td_avg_diff
0,UFC Fight Night: Ribas vs. Namajunas,Amanda Ribas,Rose Namajunas,Blue,Women's Flyweight,0,Women,Decision - Unanimous,5,5.0,...,0.00,2.54,0.94,-0.11,-0.01,0.04,-0.02,0.26,0.2,0.69
1,UFC Fight Night: Ribas vs. Namajunas,Karl Williams,Justin Tafa,Red,Heavyweight,0,Men,Decision - Unanimous,3,3.0,...,-13.16,12.70,-1.22,-3.32,-0.02,0.50,0.13,0.50,0.2,4.75
2,UFC Fight Night: Ribas vs. Namajunas,Edmen Shahbazyan,AJ Dobson,Red,Middleweight,0,Men,KO/TKO,1,3.0,...,0.00,-2.54,-0.69,-1.22,0.06,-0.37,-0.01,-0.02,0.3,0.57
3,UFC Fight Night: Ribas vs. Namajunas,Payton Talbott,Cameron Saaiman,Red,Bantamweight,0,Men,KO/TKO,2,3.0,...,0.00,7.62,2.73,-0.60,0.08,-0.28,0.00,0.43,-0.2,-0.91
4,UFC Fight Night: Ribas vs. Namajunas,Billy Quarantillo,Youssef Zalal,Blue,Featherweight,0,Men,Submission,2,3.0,...,0.00,-5.08,4.48,3.84,0.07,-0.11,-0.22,0.01,-0.2,-1.04


In [4]:
df.columns

Index(['event_name', 'r_fighter', 'b_fighter', 'winner', 'weight_class',
       'is_title_bout', 'gender', 'method', 'finish_round', 'total_rounds',
       'time_sec', 'referee', 'r_kd', 'r_sig_str', 'r_sig_str_att',
       'r_sig_str_acc', 'r_str', 'r_str_att', 'r_str_acc', 'r_td', 'r_td_att',
       'r_td_acc', 'r_sub_att', 'r_rev', 'r_ctrl_sec', 'r_wins_total',
       'r_losses_total', 'r_age', 'r_height', 'r_weight', 'r_reach',
       'r_stance', 'r_SLpM_total', 'r_SApM_total', 'r_sig_str_acc_total',
       'r_td_acc_total', 'r_str_def_total', 'r_td_def_total', 'r_sub_avg',
       'r_td_avg', 'b_kd', 'b_sig_str', 'b_sig_str_att', 'b_sig_str_acc',
       'b_str', 'b_str_att', 'b_str_acc', 'b_td', 'b_td_att', 'b_td_acc',
       'b_sub_att', 'b_rev', 'b_ctrl_sec', 'b_wins_total', 'b_losses_total',
       'b_age', 'b_height', 'b_weight', 'b_reach', 'b_stance', 'b_SLpM_total',
       'b_SApM_total', 'b_sig_str_acc_total', 'b_td_acc_total',
       'b_str_def_total', 'b_td_def_total', 'b_

In [5]:
#Found null values after running the reverse order of df
df = df.dropna()

In [6]:
df[['finish_round', 'total_rounds', 'time_sec']]

,finish_round,total_rounds,time_sec
0,5,5.0,300
1,3,3.0,300
2,1,3.0,273
3,2,3.0,21
4,2,3.0,110
...,...,...,...
7184,3,3.0,300
7207,5,5.0,300
7250,1,1.0,720
7263,1,1.0,44


In [7]:
#Finding the target values
df['target'] = (df['finish_round'] -1)*300 + df['time_sec']

In [8]:
# Preparing df to be in order to prepare the train, valid, and test datasets
df = df.iloc[::-1]

In [9]:
df = df.reset_index()
df = df.drop(columns = ['index'], axis = 1)


#2. Feature Engineering

In [10]:
old_df = df[:]

In [11]:
#Found out the career stats of each fighter is constant in all their fights which isn't true. Also found out other key stats were missing.

old_df[old_df['r_fighter'] == 'Charles Oliveira'][['r_SLpM_total', 'r_SApM_total', 'r_sig_str_acc_total',
       'r_td_acc_total', 'r_str_def_total', 'r_td_def_total', 'r_sub_avg',
       'r_td_avg']]




,r_SLpM_total,r_SApM_total,r_sig_str_acc_total,r_td_acc_total,r_str_def_total,r_td_def_total,r_sub_avg,r_td_avg
1733,3.54,3.19,0.53,0.4,0.51,0.55,2.7,2.32
2470,3.54,3.19,0.53,0.4,0.51,0.55,2.7,2.32
3298,3.54,3.19,0.53,0.4,0.51,0.55,2.7,2.32
3660,3.54,3.19,0.53,0.4,0.51,0.55,2.7,2.32
3818,3.54,3.19,0.53,0.4,0.51,0.55,2.7,2.32
3987,3.54,3.19,0.53,0.4,0.51,0.55,2.7,2.32
4260,3.54,3.19,0.53,0.4,0.51,0.55,2.7,2.32
4922,3.54,3.19,0.53,0.4,0.51,0.55,2.7,2.32
5217,3.54,3.19,0.53,0.4,0.51,0.55,2.7,2.32
5414,3.54,3.19,0.53,0.4,0.51,0.55,2.7,2.32


In [12]:
#Creating new variables and replacing the names of the columns to standardize the naming format

df['r_ctrl_sec_total'] = df['r_ctrl_sec'][:]
df['b_ctrl_sec_total'] = df['b_ctrl_sec'][:]
df['r_kd_total'] = df['r_kd'][:]
df['b_kd_total'] = df['b_kd'][:]
df['r_sig_str_ab'] = df['b_sig_str'][:]
df['b_sig_str_ab'] = df['r_sig_str'][:]
df['r_sub_att_total'] = df['r_sub_att'][:]
df['b_sub_att_total'] = df['b_sub_att'][:]
df['r_td_att_total'] = df['r_td_att'][:]
df['b_td_att_total'] = df['b_td_att'][:]
df['r_str_def'] = df['b_sig_str_att'] - df['b_sig_str']
df['b_str_def'] = df['r_sig_str_att'] - df['r_sig_str']
df['r_td_def'] = df['b_td_att'] - df['b_td']
df['b_td_def'] = df['r_td_att'] - df['r_td']
dct = {'r_SLpM_total': 'r_sig_str_total','b_SLpM_total': 'b_sig_str_total','r_SApM_total': 'r_sig_ab_total','b_SApM_total': 'b_sig_ab_total','r_td_avg': 'r_td_total','b_td_avg': 'b_td_total'}
df[[val for val in dct.values()]] = df[[key for key in dct.keys()]]
df = df.drop(columns = [key for key in dct.keys()])

In [13]:
#Creating 2d lists with the 2 variables in each list, the first variable is used to calculate the value of the second variable

red_features = [['r_kd', 'r_kd_total'], ['r_sig_str', 'r_sig_str_total'], ['r_sig_str_ab', 'r_sig_str_ab_total'], ['r_td_att', 'r_td_att_total'], ['r_sub_att', 'r_sub_att_total'], ['r_ctrl_sec', 'r_ctrl_sec_total'], ['r_sig_str_acc', 'r_sig_str_acc_total'], ['r_td_acc', 'r_td_acc_total'], ['r_td_def', 'r_td_def_total'], ['r_str_def', 'r_str_def_total']]
blue_features = [['b_kd', 'b_kd_total'], ['b_sig_str', 'b_sig_str_total'], ['b_sig_str_ab', 'b_sig_str_ab_total'], ['b_td_att', 'r_td_att_total'], ['b_sub_att', 'b_sub_att_total'], ['b_ctrl_sec', 'b_ctrl_sec_total'], ['b_sig_str_acc', 'b_sig_str_acc_total'], ['b_td_acc', 'b_td_acc_total'], ['b_td_def', 'b_td_def_total'], ['b_str_def', 'b_str_def_total']]

In [14]:
#Method to re-evaluate the stats accurate to each fighter

def accurate_stats(dataframe, r_features, b_features, fighters):

  assert len(r_features) == len(b_features)

  for fighter in fighters:
    fighter_red, fighter_blue = dataframe[dataframe['r_fighter'] == fighter], dataframe[dataframe['b_fighter'] == fighter]
    indices = pd.concat([fighter_red, fighter_blue]).sort_index().index
    feature_length = len(r_features)

    for n in np.arange(feature_length):
        count = 0
        average = 0
        sum = 0
        for index in indices:
          count += 1
          if dataframe['r_fighter'].loc[index] == fighter:

            try:
              average = sum / (count - 1)
            except ZeroDivisionError:
              average = 0
            sum += dataframe.loc[index, r_features[n][0]]
            if count == 1:
              dataframe.loc[index, r_features[n][1]] = dataframe.loc[index, r_features[n][0]]
            else:
              dataframe.loc[index, r_features[n][1]] = average
          else:

            try:
              average = sum / (count - 1)
            except ZeroDivisionError:
              average = 0
            sum += dataframe.loc[index, b_features[n][0]]
            if count == 1:
              dataframe.loc[index, b_features[n][1]] = dataframe.loc[index, b_features[n][0]]
            else:
              dataframe.loc[index, b_features[n][1]] = average

  return dataframe






In [15]:
fighters_array = np.unique(np.concatenate([np.unique(df['r_fighter']), np.unique(df['b_fighter'])]))
len(fighters_array)

1812

In [16]:
#Fixing the data

df = accurate_stats(df, red_features, blue_features, fighters_array)

/tmp/ipython-input-666399315.py:28: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '1.5' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  dataframe.loc[index, r_features[n][1]] = average
/tmp/ipython-input-666399315.py:39: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '116.33333333333333' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  dataframe.loc[index, b_features[n][1]] = average
/tmp/ipython-input-666399315.py:39: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.5' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  dataframe.loc[index, b_features[n][1]] = average
/tmp/ipython-input-666399315.py:28: FutureWarning: Setting an

In [17]:
#Creating the differnce metrics

red_totals = [feature[1] for feature in red_features]
blue_totals = [feature[1] for feature in blue_features]
final_features = [feature.strip('r_').replace('total', 'diff') for feature in red_totals]

for n in np.arange(len(final_features)):
  df[final_features[n]] = np.abs(df[red_totals[n]] - df[blue_totals[n]])

In [20]:
#Creating X and y dataframes for three round and five round fights

df['reach_diff'] = np.abs(df['reach_diff'])

x_features = red_totals + blue_totals + final_features + ['reach_diff'] + ['total_rounds']
y_features = ['target', 'total_rounds']

X, y = df[x_features], df[y_features]


three_rnd_X, five_rnd_X = X[X['total_rounds'] == 3.0].drop(['total_rounds'], axis = 1), X[X['total_rounds'] == 5.0].drop(['total_rounds'], axis = 1)
three_rnd_y, five_rnd_y = y[y['total_rounds'] == 3.0].drop(['total_rounds'], axis = 1), y[y['total_rounds'] == 5.0].drop(['total_rounds'], axis = 1)





In [21]:
#No null values found in the dataframes

three_rnd_X.info(), five_rnd_X.info(), three_rnd_y.info(), five_rnd_y.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5733 entries, 4 to 6359
Data columns (total 31 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   r_kd_total           5733 non-null   float64
 1   r_sig_str_total      5733 non-null   float64
 2   r_sig_str_ab_total   5733 non-null   float64
 3   r_td_att_total       5733 non-null   float64
 4   r_sub_att_total      5733 non-null   float64
 5   r_ctrl_sec_total     5733 non-null   float64
 6   r_sig_str_acc_total  5733 non-null   float64
 7   r_td_acc_total       5733 non-null   float64
 8   r_td_def_total       5733 non-null   float64
 9   r_str_def_total      5733 non-null   float64
 10  b_kd_total           5733 non-null   float64
 11  b_sig_str_total      5733 non-null   float64
 12  b_sig_str_ab_total   5733 non-null   float64
 13  r_td_att_total       5733 non-null   float64
 14  b_sub_att_total      5733 non-null   float64
 15  b_ctrl_sec_total     5733 non-null   float6

(None, None, None, None)